In [ ]:
!pip install pandas
!pip install geopandas
!pip install numpy

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np

Второй этап. Получение вероятностей

In [2]:
# загрузка уже построенной матрицы расстояний
df = pd.read_csv('provision/distance_matrix.csv')
print(df)

     2173828  2414213  3436636  3958429  3972447  4649424  4649425  4652381  \
0     6535.4  11829.2  10774.9  10528.7  10369.4  10233.0  10258.5   9872.2   
1    17213.9   6680.0   2559.9   4250.4   4428.4   4697.1   4560.8   2305.4   
2    17528.5   7386.3   3266.2   4956.7   5134.7   5403.4   5267.1   2246.6   
3    38575.7  32959.7  28736.8  30422.8  30600.8  30869.5  30733.2  27237.7   
4    10057.8   7763.7   7351.6   6588.7   6429.4   6293.0   6318.5   6482.2   
..       ...      ...      ...      ...      ...      ...      ...      ...   
149  14889.4   4570.9   1555.3   1572.0   1693.9   1875.0   1741.6   1921.7   
150  14535.8   5609.2   2586.7   2679.7   2798.3   2913.0   2812.5   1085.0   
151   3821.5  16602.6  15856.0  15609.8  15450.5  15314.1  15339.6  14953.3   
152  15572.8   3501.6   2006.7    891.8    783.2    805.4    704.9   3122.9   
153  22204.1   8420.0   6584.6   7748.4   7926.4   8150.4   8028.1   7479.2   

     5991849  9371321  ...  1138559754  1144248689 

In [3]:
# Заменяем NaN значения на ноль, чтобы они не влияли на расчет
df = df.fillna(0)

# Заменяем все расстояния на (1 / расстояние)
df = df.apply(lambda x: 1 / x if x.sum() > 0 else x, axis=1)

# Заменяем бесконечные значения (результат деления на 0) на NaN
df = df.replace(np.inf, np.nan)

print(df)


      2173828   2414213   3436636   3958429   3972447   4649424   4649425  \
0    0.000153  0.000085  0.000093  0.000095  0.000096  0.000098  0.000097   
1    0.000058  0.000150  0.000391  0.000235  0.000226  0.000213  0.000219   
2    0.000057  0.000135  0.000306  0.000202  0.000195  0.000185  0.000190   
3    0.000026  0.000030  0.000035  0.000033  0.000033  0.000032  0.000033   
4    0.000099  0.000129  0.000136  0.000152  0.000156  0.000159  0.000158   
..        ...       ...       ...       ...       ...       ...       ...   
149  0.000067  0.000219  0.000643  0.000636  0.000590  0.000533  0.000574   
150  0.000069  0.000178  0.000387  0.000373  0.000357  0.000343  0.000356   
151  0.000262  0.000060  0.000063  0.000064  0.000065  0.000065  0.000065   
152  0.000064  0.000286  0.000498  0.001121  0.001277  0.001242  0.001419   
153  0.000045  0.000119  0.000152  0.000129  0.000126  0.000123  0.000125   

      4652381   5991849   9371321  ...  1138559754  1144248689  1144248690 

Определяем вероятности попадания жителей каждого дома в каждую из школ 

In [4]:
# Заменяем NaN значения на нули
probabilities_df = df.fillna(0)

# Суммируем расстояния для каждого здания (по столбцам)
sum_distances_per_building = probabilities_df.sum(axis=0)

# Распределяем вероятности (делим коэффициенты на их сумму для каждого здания)
probabilities_df_normalized = probabilities_df.divide(sum_distances_per_building, axis=1)

print(probabilities_df_normalized)

      2173828   2414213   3436636   3958429   3972447   4649424   4649425  \
0    0.010759  0.002918  0.002405  0.002655  0.002752  0.002885  0.002806   
1    0.004085  0.005167  0.010121  0.006576  0.006444  0.006286  0.006311   
2    0.004011  0.004673  0.007933  0.005639  0.005557  0.005464  0.005465   
3    0.001823  0.001047  0.000902  0.000919  0.000932  0.000956  0.000937   
4    0.006991  0.004446  0.003524  0.004242  0.004438  0.004692  0.004555   
..        ...       ...       ...       ...       ...       ...       ...   
149  0.004723  0.007552  0.016659  0.017779  0.016846  0.015746  0.016527   
150  0.004837  0.006154  0.010017  0.010430  0.010197  0.010135  0.010234   
151  0.018400  0.002079  0.001634  0.001790  0.001847  0.001928  0.001876   
152  0.004515  0.009858  0.012912  0.031340  0.036434  0.036657  0.040832   
153  0.003167  0.004100  0.003935  0.003607  0.003600  0.003622  0.003585   

      4652381   5991849   9371321  ...  1138559754  1144248689  1144248690 

Проверка сумм вероятности на равенство единице

In [5]:
# Подсчет суммы по каждому зданию (столбцу)
summm = probabilities_df_normalized.sum(axis=0)

print(summm)


2173828       1.0
2414213       1.0
3436636       1.0
3958429       1.0
3972447       1.0
             ... 
1150785208    1.0
1150785209    1.0
1150785213    1.0
1150785217    1.0
1150785218    1.0
Length: 32579, dtype: float64


In [6]:
probabilities_df_normalized.to_csv('task5\probabilities_matrix.csv', index=False)

Третий этап. Распределение населения каждого здания по школам

In [3]:
houses = gpd.read_file("data/buildings.geojson")
print(houses)

               id      function  is_living  storeys_count  basement_area  \
0           28470        school      False              3       2393.244   
1           28480        school      False              3       2346.316   
2          290869        school      False              3       2337.493   
3          299980        school      False              3       2377.855   
4          300873        school      False              3       2769.566   
...           ...           ...        ...            ...            ...   
62803  1150785208    apartments       True              9       1413.145   
62804  1150785209    apartments       True              9       1987.830   
62805  1150785213  construction       True             16       1118.595   
62806  1150785217  construction       True             16        380.318   
62807  1150785218  construction       True             16        310.757   

       total_area  population  living_area  block_id  \
0             NaN           0  

In [4]:
prob_matrix = pd.read_csv('provision/probabilities_matrix.csv')
print(prob_matrix)

     2173828  2414213  3436636   3958429   3972447  4649424   4649425  \
0        0.0      0.0      0.0  0.000000  0.000000      0.0  0.000000   
1        0.0      0.0      0.0  0.000000  0.000000      0.0  0.000000   
2        0.0      0.0      0.0  0.000000  0.000000      0.0  0.000000   
3        0.0      0.0      0.0  0.000000  0.000000      0.0  0.000000   
4        0.0      0.0      0.0  0.000000  0.000000      0.0  0.000000   
..       ...      ...      ...       ...       ...      ...       ...   
149      0.0      0.0      0.0  0.000000  0.000000      0.0  0.000000   
150      0.0      0.0      0.0  0.000000  0.000000      0.0  0.000000   
151      0.0      0.0      0.0  0.000000  0.000000      0.0  0.000000   
152      0.0      0.0      0.0  0.394497  0.312281      1.0  0.416509   
153      0.0      0.0      0.0  0.000000  0.000000      0.0  0.000000   

     4652381  5991849   9371321  ...  1138559754  1144248689  1144248690  \
0        0.0      0.0  0.000000  ...         0.

In [6]:
houses.index = houses['id']
print(houses)

                    id      function  is_living  storeys_count  basement_area  \
id                                                                              
28470            28470        school      False              3       2393.244   
28480            28480        school      False              3       2346.316   
290869          290869        school      False              3       2337.493   
299980          299980        school      False              3       2377.855   
300873          300873        school      False              3       2769.566   
...                ...           ...        ...            ...            ...   
1150785208  1150785208    apartments       True              9       1413.145   
1150785209  1150785209    apartments       True              9       1987.830   
1150785213  1150785213  construction       True             16       1118.595   
1150785217  1150785217  construction       True             16        380.318   
1150785218  1150785218  cons

In [9]:
prob_matrix_copy = prob_matrix
weighted_population_matrix = pd.DataFrame(index=prob_matrix_copy.index, columns=prob_matrix_copy.columns)

# Умножаем населения зданий на вероятности попадания в каждую школу
for column_name, column_data in prob_matrix_copy.iteritems():
    pop = houses.loc[int(column_name), 'population']
    weighted_population_matrix[column_name] = column_data * pop

print(weighted_population_matrix)

C:\Users\sm_pa\AppData\Local\Temp\ipykernel_7472\887165592.py:4: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column_name, column_data in prob_matrix_copy.iteritems():


     2173828  2414213  3436636     3958429     3972447  4649424    4649425  \
0        0.0      0.0      0.0    0.000000    0.000000      0.0    0.00000   
1        0.0      0.0      0.0    0.000000    0.000000      0.0    0.00000   
2        0.0      0.0      0.0    0.000000    0.000000      0.0    0.00000   
3        0.0      0.0      0.0    0.000000    0.000000      0.0    0.00000   
4        0.0      0.0      0.0    0.000000    0.000000      0.0    0.00000   
..       ...      ...      ...         ...         ...      ...        ...   
149      0.0      0.0      0.0    0.000000    0.000000      0.0    0.00000   
150      0.0      0.0      0.0    0.000000    0.000000      0.0    0.00000   
151      0.0      0.0      0.0    0.000000    0.000000      0.0    0.00000   
152      0.0      0.0      0.0  127.028046  340.698246    418.0  175.76684   
153      0.0      0.0      0.0    0.000000    0.000000      0.0    0.00000   

     4652381  5991849     9371321  ...  1138559754  1144248689 

In [10]:
weighted_population_matrix.to_csv('task5\population_matrix.csv', index=False)

Четвертый этап: соотношение емкости сервиса и суммарного потенциального спроса на сервис

Подсчет потенциального спроса на каждую школу

In [13]:
weighted_population_matrix.fillna(0, inplace=True)

schools = gpd.read_file("data/services/school.geojson")
schools_demand = schools
schools_demand['total_population'] = 0.0

# Итерация по столбцам матрицы и суммирование населения, попавшего в каждую школу
for column_name, column_data in weighted_population_matrix.iteritems():
    schools_demand['total_population'] += column_data

print(schools_demand)

C:\Users\sm_pa\AppData\Local\Temp\ipykernel_7472\3635671213.py:8: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column_name, column_data in weighted_population_matrix.iteritems():


      full_id      osm_id  osm_type amenity operator:type    full_name  \
0      r28480       28480  relation  school          None         None   
1    r1523489     1523489  relation  school          None         None   
2    r1781858     1781858  relation  school          None         None   
3    r1861681     1861681  relation  school          None         None   
4    r8865720     8865720  relation  school          None         None   
..        ...         ...       ...     ...           ...          ...   
149      None  2469072357      None    None          None         None   
150      None  3156127068      None    None          None         None   
151      None  4809406521      None    None          None         None   
152      None  6364071785      None    None          None         None   
153      None        None      None  school          None  Гимназия №1   

    source:addr religion addr:place ownership  ...  foot locked barrier_2  \
0          None     None       Non

In [14]:
# Расчет коэффициента загруженности для каждой школы
schools_demand['provision_rate'] = schools_demand['Children'] / schools_demand['total_population']
print(schools_demand)

      full_id      osm_id  osm_type amenity operator:type    full_name  \
0      r28480       28480  relation  school          None         None   
1    r1523489     1523489  relation  school          None         None   
2    r1781858     1781858  relation  school          None         None   
3    r1861681     1861681  relation  school          None         None   
4    r8865720     8865720  relation  school          None         None   
..        ...         ...       ...     ...           ...          ...   
149      None  2469072357      None    None          None         None   
150      None  3156127068      None    None          None         None   
151      None  4809406521      None    None          None         None   
152      None  6364071785      None    None          None         None   
153      None        None      None  school          None  Гимназия №1   

    source:addr religion addr:place ownership  ... locked barrier_2 access  \
0          None     None       No

Для реалистичности умножаем население на 0.12 - в Перми школьники составляют 12% от населения города

In [16]:
schools_demand['total_population'] = schools_demand['total_population'] * 0.12
schools_demand['provision_rate'] = schools_demand['provision_rate'] / 0.12
print(schools_demand)

      full_id      osm_id  osm_type amenity operator:type    full_name  \
0      r28480       28480  relation  school          None         None   
1    r1523489     1523489  relation  school          None         None   
2    r1781858     1781858  relation  school          None         None   
3    r1861681     1861681  relation  school          None         None   
4    r8865720     8865720  relation  school          None         None   
..        ...         ...       ...     ...           ...          ...   
149      None  2469072357      None    None          None         None   
150      None  3156127068      None    None          None         None   
151      None  4809406521      None    None          None         None   
152      None  6364071785      None    None          None         None   
153      None        None      None  school          None  Гимназия №1   

    source:addr religion addr:place ownership  ... locked barrier_2 access  \
0          None     None       No

In [17]:
schools_demand.to_file('task5/school_demand.geojson', driver='GeoJSON')

5 этап: Суммирование коэффициентов обеспеченности для каждого здания

In [ ]:
distance_matrix = pd.read_csv('task5/distance_matrix.csv')

In [ ]:
# Задаем радиус доступности
radius = 1000

buildings_provision = houses[houses['is_living'] == 1]

buildings_provision['total_provision_rate'] = 0.0

# Итерация по зданиям
for building_index, building_row in buildings_provision.iterrows():
    building_index_str = str(building_index)

    # Выбор школ, которые попадают в радиус доступности
    schools_in_radius = distance_matrix.loc[distance_matrix[building_index_str] <= radius].index

    # Суммирование коэффициентов обеспеченности для выбранных школ
    total_provision_rate = schools_demand.loc[schools_in_radius, 'provision_rate'].sum()
    
    # Присвоение суммы коэффициентов в новый столбец
    buildings_provision.at[building_index, 'total_provision_rate'] = total_provision_rate

In [31]:
print(buildings_provision)

                    id      function  is_living  storeys_count  basement_area  \
id                                                                              
2173828        2173828           yes       True              7       2405.573   
2414213        2414213           yes       True              3       1342.551   
3436636        3436636           yes       True              1        176.517   
3958429        3958429           yes       True             10       1813.652   
3972447        3972447    apartments       True             25       2413.605   
...                ...           ...        ...            ...            ...   
1150785208  1150785208    apartments       True              9       1413.145   
1150785209  1150785209    apartments       True              9       1987.830   
1150785213  1150785213  construction       True             16       1118.595   
1150785217  1150785217  construction       True             16        380.318   
1150785218  1150785218  cons

In [33]:
buildings_provision = buildings_provision.reset_index(drop=True)
buildings_provision.to_file('task5/buildings_provision.geojson', driver='GeoJSON')

Сохранение результатов

In [5]:
schools = gpd.read_file("data/services/school.geojson")
buildings_provision = gpd.read_file("task5/buildings_provision.geojson")